In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.rnn as rnn_utils
import polars as pl
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
from typing import Dict, List, Tuple

In [ ]:
# Constants
BATCH_SIZE = 32
EMB_DIM = 128
HID_DIM = 258
N_LAYERS = 2
DROPOUT = 0.5
LEARNING_RATE = 0.001
EPOCHS = 5
FRAC = 0.2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load parquet file
local_path = "../data/output/processed.parquet"
colab_path = "../content/processed.parquet"
loaded_data = pl.read_parquet(colab_path)
print("Data loaded successfully")
loaded_data.head()

In [ ]:
input_data = loaded_data.sample(fraction = FRAC)

In [ ]:
input_data.describe()

In [ ]:
def preprocess_data(df):
    it = [torch.tensor(tokens) for tokens in df["it"]]
    en = [torch.tensor(tokens) for tokens in df["en"]]
    return list(zip(en, it))

In [ ]:
data_pairs = preprocess_data(input_data)

In [ ]:
EN_VOCAB_SIZE = max(max(seq) for seq, _ in data_pairs) + 1
IT_VOCAB_SIZE = max(max(seq) for _, seq in data_pairs) + 1

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.pairs[idx]

In [ ]:
def collate_custom(batch):
    src = [pair[0] for pair in batch]
    trg = [pair[1] for pair in batch]
    src_padded = nn.utils.rnn.pad_sequence(src, batch_first=True, padding_value=0)
    trg_padded = nn.utils.rnn.pad_sequence(trg, batch_first=True, padding_value=0)
    return src_padded, trg_padded

In [ ]:
train_loader = DataLoader(
    TranslationDataset(data_pairs),
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_custom
)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, hidden, cell):
        trg = trg.unsqueeze(1)
        embedded = self.dropout(self.embedding(trg))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc_out.out_features

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        trg_input = trg[:, 0]
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(trg_input, hidden, cell)
            outputs[:, t, :] = output
            top1 = output.argmax(1)
            trg_input = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

In [ ]:
encoder = Encoder(IT_VOCAB_SIZE, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
decoder = Decoder(EN_VOCAB_SIZE, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0

    for src, trg in iterator:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()

        output = model(src, trg)
        output_dim = output.shape[-1]

        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)

        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for src, trg in iterator:
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0)  # Turn off teacher forcing
            output_dim = output.shape[-1]

            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [ ]:
train_losses, eval_losses = [], []

for epoch in range(EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion)
    eval_loss = evaluate(model, train_loader, criterion)

    train_losses.append(train_loss)
    eval_losses.append(eval_loss)

    print(f"Epoch {epoch + 1}/{EPOCHS} - Train Loss: {train_loss:.4f} | Eval Loss: {eval_loss:.4f}")


In [ ]:
plt.plot(range(1, EPOCHS + 1), train_losses, label="Train Loss")
plt.plot(range(1, EPOCHS + 1), eval_losses, label="Eval Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Training and Evaluation Loss")
plt.show()
